# Data loading 
Next cells are used to load the data we need.

In [ ]:
!git clone #clone here your repository
!mv Recommender-Systems-Challenge/* ../content/

Cloning into 'Recommender-Systems-Challenge'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 1063 (delta 79), reused 79 (delta 44), pack-reused 937
Receiving objects: 100% (1063/1063), 208.24 MiB | 42.40 MiB/s, done.
Resolving deltas: 100% (617/617), done.


In [ ]:
!python run_compile_all_cython.py

run_compile_all_cython: Found 11 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/usr/bin/python3'
Compiling [1/11]: MatrixFactorization_Cython_Epoch.pyx... 
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/MatrixFactorization/Cython/MatrixFactorization_Cython_Epoch.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
In file included from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1822:0,
                 from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from MatrixFactorization_Cython_Epoch.c:610:
/usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/npy_1_7_d

In [ ]:
pip install implicit

     |████████████████████████████████| 1.1MB 15.5MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419457 sha256=28ec196fbba28f652a10ac1cd40ba8735a53fd935487eea5b93c502d76e8b40f
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [ ]:
pip install optuna

     |████████████████████████████████| 266kB 16.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163kB 39.2MB/s 
     |████████████████████████████████| 81kB 11.4MB/s 
     |████████████████████████████████| 81kB 11.8MB/s 
     |████████████████████████████████| 112kB 19.0MB/s 
     |████████████████████████████████| 133kB 29.9MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
  Created wheel for optuna: filename=optuna-2.3.0-cp36-none-any.whl size=359761 sha256=fb18f0dfa9d04910a7f0f2eea707daaaa495e898eded47ae2bcec4a1f3927085
  Stored in directory: /root/.cache/pip/wheels/fa/91/19/64b0ec6b964f89c0695a9dc6db6f851d0b54c5381a5c9cadfb
Successfully built optuna
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any.whl size=13702 sha256=7f2ed37bafde4033839c9eef479d7612af8b77f047ee925815f38274d2b2e4d9
  Stored in directory: /root/.cache/pip/

In [ ]:
import os
import pandas as pd
import numpy as np 
import scipy.sparse as sps
import sys
import matplotlib.pyplot as plt
import optuna


from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from MatrixFactorization.IALSRecommender_implicit import IALSRecommender_implicit

In [ ]:
dataset = pd.read_csv('/content/New Challenge Data/data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


# Data pre-processing
Pre-processing of data to check for missing users, items ecc...

In [ ]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

test_users = pd.read_csv('/content/New Challenge Data/data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
7939,7942
7940,7943
7941,7944
7942,7945


In [ ]:
ICM_df = pd.read_csv('/content/New Challenge Data/data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [ ]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [ ]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Base.Evaluation.Evaluator import EvaluatorHoldout

URM_train_1, URM_validation_1 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 1)

URM_train_2, URM_validation_2 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 2)

URM_train_3, URM_validation_3 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 3)

URM_train_4, URM_validation_4 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 4)

URM_train_5, URM_validation_5 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 5)

URM_train_vector = [URM_train_1,URM_train_2,URM_train_3,URM_train_4,URM_train_5]

evaluator_validation_1 = EvaluatorHoldout(URM_validation_1, cutoff_list=[10])
evaluator_validation_2 = EvaluatorHoldout(URM_validation_2, cutoff_list=[10])
evaluator_validation_3 = EvaluatorHoldout(URM_validation_3, cutoff_list=[10])
evaluator_validation_4 = EvaluatorHoldout(URM_validation_4, cutoff_list=[10])
evaluator_validation_5 = EvaluatorHoldout(URM_validation_5, cutoff_list=[10])

Evaluator_vector = [evaluator_validation_1, evaluator_validation_2, evaluator_validation_3, evaluator_validation_4, evaluator_validation_5]

In [ ]:
class Objective(object):
    def __init__(self, URM_train_vector, ICM_all, Evaluator_vector):
        # Hold this implementation specific arguments as the fields of the class.
        self.URM_train_vector = URM_train_vector
        self.ICM_all = ICM_all
        self.Evaluator_vector = Evaluator_vector
        self.MAP_vector = np.zeros(5)

        

    def __call__(self, trial):
        # Calculate an objective value by using the extra arguments.
        n_factors = trial.suggest_int('n_factors', 500,900)
        regularization = trial.suggest_uniform('regularization', 0.4,0.8)
        iterations = trial.suggest_int('iterations', 60, 90)
        weight = trial.suggest_uniform('weight', 0.4, 0.65)
        self.MAP_vector = np.zeros(5)
        i = 0        
        for URM_train, evaluator_validation in zip(self.URM_train_vector,self.Evaluator_vector):
            URM_train_ICM_all = sps.vstack([URM_train*(1-weight), ICM_all.T*weight])
            recommender = IALSRecommender_implicit(URM_train_ICM_all)
            recommender.fit(n_factors = n_factors, regularization = regularization, iterations = iterations)
            result_dict, _ = evaluator_validation.evaluateRecommender(recommender)
            self.MAP_vector[i]=result_dict[10]["MAP"]
            i = i+1
            
        print('printing self map vector: ', self.MAP_vector)
        MAP = np.sum(self.MAP_vector)/5
        print('printing MAP: ', MAP)
        
        return MAP
    
# Execute an optimization by using an `Objective` instance.
study = optuna.create_study(direction='maximize')
study.optimize(Objective(URM_train_vector, ICM_all, Evaluator_vector), n_trials=50)

print(study.best_params)

[I 2021-01-04 10:20:05,547] A new study created in memory with name: no-name-cc8c7535-59ca-48cf-8d3a-96ce8a88388b


Recommender_Base_Class: URM Detected 73 (0.26 %) cold users.
Recommender_Base_Class: URM Detected 1 (0.00 %) cold items.



EvaluatorHoldout: Processed 5536 ( 100.00% ) in 6.14 sec. Users per second: 902


Recommender_Base_Class: URM Detected 59 (0.21 %) cold users.
Recommender_Base_Class: URM Detected 1 (0.00 %) cold items.



EvaluatorHoldout: Processed 5627 ( 100.00% ) in 6.07 sec. Users per second: 927


...


EvaluatorHoldout: Processed 5674 ( 100.00% ) in 5.42 sec. Users per second: 1047


Recommender_Base_Class: URM Detected 66 (0.24 %) cold users.
Recommender_Base_Class: URM Detected 3 (0.01 %) cold items.



EvaluatorHoldout: Processed 5589 ( 100.00% ) in 5.16 sec. Users per second: 1083


[I 2021-01-04 15:13:29,116] Trial 49 finished with value: 0.0730445533780579 and parameters: {'n_factors': 503, 'regularization': 0.5823719483474717, 'iterations': 70, 'weight': 0.5751062684435179}. Best is trial 24 with value: 0.07579190538757033.


printing self map vector:  [0.07841483 0.07204805 0.07292207 0.07133502 0.07050279]
printing MAP:  0.0730445533780579
{'n_factors': 768, 'regularization': 0.4489004525533907, 'iterations': 76, 'weight': 0.5293971458364172}
